# Probability of having a sister

https://platform.stratascratch.com/technical/2368-probability-of-having-a-sister?tabname=question

Assume the distribution of children per family is as given in the table below:


| n_children | 0   | 1    | 2   | 3    | 4   | >=5 |
|------------|-----|------|-----|------|-----|-----|
| p          | 0.3 | 0.25 | 0.2 | 0.15 | 0.1 | 0   |

Consider a random girl in the population of children. What is the probability that she has a sister?